# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k
#from utils import prefilter_items

In [4]:
data = pd.read_csv(r'C:\\Users\a.slivaev\Desktop\webinar_2\retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
item_features = pd.read_csv(r'C:\\Users\a.slivaev\Desktop\webinar_2\product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\a.slivaev\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\a.slivaev\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Decreased # items from 86865 to 2690


In [8]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,420647,545926,707683,819063,819227,819255,819304,819308,819330,...,14077333,14106445,15452677,15596488,15596515,15831255,15926844,15926886,15927403,15927850
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [11]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 1.16 s


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

----

Проверка, что все работает

In [2]:
from metrics import precision_at_k, recall_at_k
#from utils import prefilter_items
#from recommenders import MainRecommender



In [13]:
def prefilter_items(data_train, item_features):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
     
    # Уберем самые популярные
    top_corrected = top_5000[10:]
    
    # Уберем самые непопулярные
    top_corrected = top_corrected[:-10]
    data_train.loc[~data_train['item_id'].isin(top_corrected), 'item_id'] = 999999 
    
    # Уберем товары, которые не продавались за последние 12 месяцев (это примерно 52 недели)
    data_train.loc[data_train['week_no'] < data_train['week_no'].max() - 52, 'item_id'] = 999999  # Оставляю продажи после 39 недели
    
    # Уберем не интересные для рекоммендаций категории (department)
    data_train_dep = pd.merge(data_train, item_features[['item_id', 'department']], how='left', on="item_id")
    bad_departments = ['KIOSK-GAS']
    for dept in bad_departments:
        data_train_dep.loc[data_train_dep['department'] == dept, 'item_id'] = 999999
    data_train = data_train_dep.drop(columns=['department'])

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data_train.loc[data_train['sales_value'] < 6, 'item_id'] = 999999
    
    # Уберем слишком дорогие товары
    data_train.loc[data_train['sales_value'] > 500, 'item_id'] = 999999

    return data_train

def postfilter_items():
    pass

In [14]:
popularity = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
popularity.sort_values('quantity', ascending=False, inplace=True)  # "Популярность" по кол-ву покупок

item_filter = 999999
if item_filter:
    popularity = popularity[popularity['item_id'] != item_filter]  # item_filter = 999999, dummy item

popularity = popularity.groupby('user_id')

In [15]:
own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

own.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

In [16]:
def get_rec(model, x):

    recs = model.similar_items(itemid_to_id[x], N=3)
    top_rec = recs[1][0] if id_to_itemid[recs[1][0]] != 999999 else recs[2][0]

    return id_to_itemid[top_rec]

def get_recommendations(user, model, sparse_user_item, N=5):
    """Рекомендуем топ-N товаров"""
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! Используем фильтр
                                    recalculate_user=True)]
    return res

def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

    popularity_n_items = popularity.head(N)  # Оставляем первые N самых "популярных" товаров
    res = popularity_n_items[popularity_n_items['user_id'] == user]['item_id'].apply(lambda x: get_rec(model, x))  # Результат для конкретного юзера

    res = res.tolist()

    #  Пока закостылю, надо подумать, при user=10 только 1 рекомендация
    if len(res) < N:
        for i in range(N - len(res)):
            res.append(999999)
   
    return res

def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    res = []
    try:
        for matrix_userid in model.similar_users(userid_to_id[user], N=N+1):
            if matrix_userid[0] != userid_to_id[user]:
                if len(get_recommendations(user=id_to_userid[matrix_userid[0]], model=own, sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=1)) != 0:
                    res.append(get_recommendations(user=id_to_userid[matrix_userid[0]], model=own, sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=1)[0])
                else:
                    res.append(999999)
    except:
        res = [999999 for i in range(N)]  # Потом поправлю, ошибка для user=2325 KeyError, запусти userid_to_id[2325]

    res = res[:N] if len(res) > N else res  # См. 102 - 6 рекомендаций

    return res

Проверим ниже работу функций get_similar_items_recommendation и get_similar_users_recommendation

In [17]:
get_similar_items_recommendation(user=10, model=model, N=5)

[1026011, 999999, 999999, 999999, 999999]

In [18]:
get_similar_users_recommendation(user=102, model=model, N=5)

[12782182, 9553397, 941883, 965772, 5569792]

In [19]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, item_features, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        data = self.prefilter_items(data, item_features)

        # predefined code
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

        # вынес сюда группировку, чтобы не делать её каждый раз при вызове метода get_similar_items_recommendation
        self.popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.popularity.sort_values('quantity', ascending=False, inplace=True)  # "Популярность" по кол-ву покупок

        item_filter = 999999
        if item_filter:
            self.popularity = self.popularity[self.popularity['item_id'] != item_filter]  # item_filter = 999999, dummy item
        self.popularity = self.popularity.groupby('user_id')  # Заранее сгруппируем по юзерам

    @staticmethod
    def prefilter_items(data_train, item_features):
        # Оставим только 5000 самых популярных товаров
        popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
        popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
        top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
        #добавим, чтобы не потерять юзеров
        data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
        
        # Уберем самые популярные
        top_corrected = top_5000[10:]
        
        # Уберем самые непопулярные
        top_corrected = top_corrected[:-10]
        data_train.loc[~data_train['item_id'].isin(top_corrected), 'item_id'] = 999999 
        
        # Уберем товары, которые не продавались за последние 12 месяцев (это примерно 52 недели)
        data_train.loc[data_train['week_no'] < data_train['week_no'].max() - 52, 'item_id'] = 999999  # Оставляю продажи после 39 недели
        
        # Уберем не интересные для рекоммендаций категории (department)
        data_train_dep = pd.merge(data_train, item_features[['item_id', 'department']], how='left', on="item_id")
        bad_departments = ['KIOSK-GAS']
        for dept in bad_departments:
            data_train_dep.loc[data_train_dep['department'] == dept, 'item_id'] = 999999
        data_train = data_train_dep.drop(columns=['department'])

        # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
        data_train.loc[data_train['sales_value'] < 6, 'item_id'] = 999999
        
        # Уберем слишком дорогие товары
        data_train.loc[data_train['sales_value'] > 500, 'item_id'] = 999999

        return data_train

    @staticmethod
    def postfilter_items():
        pass

    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,
                                             use_gpu=False,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    @staticmethod
    def get_rec(model, x):
        '''Рекомендует 1 наиболее похожий item'''

        recs = model.similar_items(itemid_to_id[x], N=3)
        top_rec = recs[1][0] if id_to_itemid[recs[1][0]] != 999999 else recs[2][0]  # Фильтрую dummy item 999999, для этого выше N=3, чтобы был альтернативный вариант

        return id_to_itemid[top_rec]

    @staticmethod
    def get_recommendations(user, model, sparse_user_item, N=5):
        """Рекомендуем топ-N товаров"""
        res = [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[user], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]],  # !!! Используем фильтр
                                        recalculate_user=True)]
        return res

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        self.popularity_n_items = self.popularity.head(N)  # Оставляем первые N самых "популярных" товаров

        try:
            res = self.popularity_n_items[self.popularity_n_items['user_id'] == user]['item_id'].apply(lambda x: get_rec(self.model, x))  # Результат для конкретного юзера

            res = res.tolist()

            #  Пока закостылю, надо подумать, при user=10 только 1 рекомендация
            if len(res) < N:
                for i in range(N - len(res)):
                    res.append(999999)
        except:
            res = [999999 for i in range(N)]  # Потом поправлю, ошибка для user=2325 KeyError, запусти userid_to_id[2325]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []
        try:
            for matrix_userid in model.similar_users(userid_to_id[user], N=N+1):
                if matrix_userid[0] != userid_to_id[user]:
                    if len(get_recommendations(user=id_to_userid[matrix_userid[0]], model=own, sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=1)) != 0:
                        res.append(get_recommendations(user=id_to_userid[matrix_userid[0]], model=own, sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=1)[0])
                    else:
                        res.append(999999)
        except:
            res = [999999 for i in range(N)]  # Потом поправлю, ошибка для user=2325 KeyError, запусти userid_to_id[2325]

        res = res[:N] if len(res) > N else res  # См. 102 - 6 рекомендаций

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [21]:
my_recomender = MainRecommender(data_train, item_features)

In [22]:
my_recomender.get_similar_items_recommendation(2500, N=5)

[10455984, 885406, 967144, 845193, 985999]

In [23]:
my_recomender.get_similar_users_recommendation(10, N=5)

[972665, 1048257, 948650, 827656, 838186]

In [24]:
result['similar_items'] = result['user_id'].apply(lambda x: my_recomender.get_similar_items_recommendation(x, N=5))

In [25]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean()

0.0177277179236043

In [26]:
result['similar_users'] = result['user_id'].apply(lambda x: my_recomender.get_similar_users_recommendation(x, N=5))

In [27]:
result.apply(lambda row: precision_at_k(row['similar_users'], row['actual']), axis=1).mean()

0.01106758080313416

Качество получилось очень низкое, но это ожидаемо, т.к. мы не предсказывали вероятные покупки, а предлагали пользователю новый опыт. Поэтому precision метрика на тестовой выборке не отражает бизнес эффект от таких рекомендаций